In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from IPython.display import IFrame

In [3]:
top100 = pd.read_csv('top100.csv')
top_10000 = pd.read_csv('song_clusters.csv')

In [ ]:
song = input("Song title: ")
song = song.strip().lower()
artist = input("Name of the Artist: ")
artist = artist.strip().lower()
user_result = sp.search(q="artist:" + artist + " track:" + song, type="track", limit=1)
tracks = json_normalize(user_result["tracks"]["items"])

In [ ]:
#identify wrong input
if user_result['tracks']['items'] == []:
    print ('Please try again')
    
elif song in top100_df['title'].str.lower().values and artist in top100_df['artist'].str.lower().values:
            #select a random song
        random=top100_df.sample()
            #searching the song in spotify to produce a clikable link
        random_result = sp.search (q="artist:" + random['artist'].values[0] + 
                                   " track:" + random['title'].values[0], type="track", limit=1)
        random_song = json_normalize(random_result["tracks"]["items"])
        song_id = random_song['id'].values[0]
        print("You should check out this hot song:")
        print(random['title'].values[0], ' by ', random['artist'].values[0])
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_id}",
                       width="320",
                       height="80",
                       frameborder="0",
                       allowtransparency="true",
                       allow="encrypted-media",
                      ))
else:
        #recommend top 10000 songs
            #extracting audio features
        audio_features_list = sp.audio_features(tracks['id'][0])
        audio_features_df = json_normalize(audio_features_list)
        user_df = pd.concat([tracks["popularity"], audio_features_df], axis=1)
        num_df=user_df.select_dtypes('number')
            #using saved scaler and cluster model
        scale = pickle.load(open('scaler_model.pkl','rb'))
        num_scaled = scale.transform(num_df)
        data_scaled = pd.DataFrame(num_scaled, columns=num_df.columns)
        model = pickle.load(open('clustering_model.pkl','rb'))
        user_cluster = model.predict(data_scaled)
            #retrieving cluster number
        user_cluster = user_cluster[0]
            #selecting songs subset from the same cluster
        songs_cluster= top_10000[top_10000["cluster"]==user_cluster]
            #recommending random song from the cluster
        random=songs_cluster.sample()
        song_id=random['song_id'].values[0]
        print("You should check out:")
        print(random['song_name'].values[0], ' by ', random['name'].values[0])
            #generate clikable link
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_id}",
                       width="320",
                       height="80",
                       frameborder="0",
                       allowtransparency="true",
                       allow="encrypted-media",
                      ))
        return